# EmoRecSys - Demographic

[EmoRecSys Survey](https://emorecsys.pt/)

###### Imports

In [1]:
# ---- INSTALLATIONS ---- #
# !pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357252 sha256=49f37c57f30c6d4c58bec5da0b3fe647c4cf72c64eca75adef4a8e3618afbfae
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [16]:
# ---- IMPORTS ---- #
from sklearn.cluster import KMeans, AgglomerativeClustering, SpectralClustering, DBSCAN
from sklearn.metrics.pairwise import euclidean_distances
from ipywidgets import HBox, VBox, Image as WidgetImage
from sklearn.model_selection import ParameterGrid
from sklearn.decomposition import PCA
from surprise import Reader, Dataset
from IPython.display import display
import pandas as pd
import numpy as np
import warnings
import random
import math

warnings.filterwarnings("ignore", message="Exited. *not reaching the requested tolerance.*", category=UserWarning)
warnings.filterwarnings("ignore", message="X does not have valid feature names, but KMeans was fitted with feature names", category=UserWarning)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1. Exploratory Data Analysis (EDA)

In [3]:
dataset_demo = pd.read_csv("../data/csvs/demographic.csv")
dataset_demo.head()

,id_survey,age,populational_aff,gender,education,city,country_residence,date_survey,consented,hobby_other,...,disease_10,disease_11,visual_acuity1,visual_acuity2,visual_acuity3,visual_acuity5,visual_acuity6,visual_acuity9,visual_acuity12,visual_acuity20
0,1,25 - 30,White/Caucasian,F,Bachelor’s Degree,Lisboa,Portugal,2024-04-10 15:32:17,1,NaN,...,0,1,1,0,0,0,0,0,0,0
1,2,25 - 30,White/Caucasian,F,Master’s Degree,Viseu,Portugal,2024-04-10 15:35:48,1,NaN,...,0,1,0,0,1,0,0,0,0,0
2,3,18 - 24,White/Caucasian,F,Master’s Degree,Santarém,Portugal,2024-04-10 16:09:37,1,NaN,...,0,1,1,0,0,0,0,0,0,0
3,4,18 - 24,White/Caucasian,M,Bachelor’s Degree,Santarém,Portugal,2024-04-10 20:37:30,1,NaN,...,0,1,0,0,0,0,0,0,0,0
4,5,41 - 50,Latino,M,Middle School,Santarém,Portugal,2024-04-10 20:42:25,1,NaN,...,0,1,0,1,0,0,0,0,0,0


In [4]:
dataset_demo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163 entries, 0 to 162
Data columns (total 45 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id_survey          163 non-null    int64 
 1   age                163 non-null    object
 2   populational_aff   163 non-null    object
 3   gender             163 non-null    object
 4   education          163 non-null    object
 5   city               163 non-null    object
 6   country_residence  163 non-null    object
 7   date_survey        163 non-null    object
 8   consented          163 non-null    int64 
 9   hobby_other        27 non-null     object
 10  hobby_1            163 non-null    int64 
 11  hobby_2            163 non-null    int64 
 12  hobby_3            163 non-null    int64 
 13  hobby_4            163 non-null    int64 
 14  hobby_5            163 non-null    int64 
 15  hobby_6            163 non-null    int64 
 16  hobby_7            163 non-null    int64 
 1

In [5]:
dataset_demo.describe()

,id_survey,consented,hobby_1,hobby_2,hobby_3,hobby_4,hobby_5,hobby_6,hobby_7,hobby_8,...,disease_10,disease_11,visual_acuity1,visual_acuity2,visual_acuity3,visual_acuity5,visual_acuity6,visual_acuity9,visual_acuity12,visual_acuity20
count,163.000000,163.0,163.000000,163.000000,163.000000,163.000000,163.000000,163.000000,163.000000,163.000000,...,163.000000,163.000000,163.000000,163.000000,163.000000,163.000000,163.000000,163.000000,163.000000,163.000000
mean,82.000000,1.0,0.104294,0.208589,0.079755,0.190184,0.067485,0.134969,0.042945,0.098160,...,0.012270,0.852761,0.300613,0.079755,0.239264,0.006135,0.012270,0.018405,0.012270,0.006135
std,47.198164,0.0,0.306584,0.407552,0.271748,0.393656,0.251633,0.342744,0.203357,0.298447,...,0.110427,0.355436,0.459938,0.271748,0.427949,0.078326,0.110427,0.134825,0.110427,0.078326
min,1.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,41.500000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,82.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,122.500000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,163.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
dataset_demo.isnull().sum()

id_survey              0
age                    0
populational_aff       0
gender                 0
education              0
city                   0
country_residence      0
date_survey            0
consented              0
hobby_other          136
hobby_1                0
hobby_2                0
hobby_3                0
hobby_4                0
hobby_5                0
hobby_6                0
hobby_7                0
hobby_8                0
hobby_9                0
hobby_10               0
hobby_11               0
hobby_12               0
hobby_13               0
hobby_14               0
hobby_15               0
hobby_16               0
hobby_17               0
hobby_18               0
disease_1              0
disease_2              0
disease_3              0
disease_4              0
disease_5              0
disease_7              0
disease_9              0
disease_10             0
disease_11             0
visual_acuity1         0
visual_acuity2         0
visual_acuity3         0


In [7]:
dataset_demo.nunique()

id_survey            163
age                    7
populational_aff       6
gender                 3
education              6
city                  36
country_residence     18
date_survey          163
consented              1
hobby_other           20
hobby_1                2
hobby_2                2
hobby_3                2
hobby_4                2
hobby_5                2
hobby_6                2
hobby_7                2
hobby_8                2
hobby_9                2
hobby_10               2
hobby_11               2
hobby_12               2
hobby_13               2
hobby_14               2
hobby_15               2
hobby_16               2
hobby_17               2
hobby_18               2
disease_1              2
disease_2              2
disease_3              2
disease_4              2
disease_5              2
disease_7              2
disease_9              2
disease_10             2
disease_11             2
visual_acuity1         2
visual_acuity2         2
visual_acuity3         2


## 2. Preprocessing

In [8]:
user_profiles = dataset_demo.iloc[:, -27:].copy()

# one-hot encoder
for col in ['hobby_other', 'country_residence', 'city', 'education', 'gender', 'populational_aff', 'age']:
  dummies = pd.get_dummies(dataset_demo[col], prefix=col)
  user_profiles = pd.concat([dummies, user_profiles], axis=1)

user_profiles.dropna(inplace=True)
user_profiles = user_profiles.applymap(lambda x: 1 if x is True else 0 if x is False else x)

user_profiles

,age_18 - 24,age_25 - 30,age_31 - 40,age_41 - 50,age_51 - 60,age_61 - 70,age_71 - 80,populational_aff_Asian,populational_aff_Black,populational_aff_Hispanic,...,disease_10,disease_11,visual_acuity1,visual_acuity2,visual_acuity3,visual_acuity5,visual_acuity6,visual_acuity9,visual_acuity12,visual_acuity20
0,0,1,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
159,0,1,0,0,0,0,0,1,0,0,...,0,1,0,0,1,0,0,0,0,0
160,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
161,0,1,0,0,0,0,0,1,0,0,...,0,1,1,0,0,0,0,0,0,0


In [9]:
## applying PCA
pca = PCA(n_components=50)
pca.fit(user_profiles)
tve = 0 # total variance explained
for i, ve in enumerate(pca.explained_variance_ratio_):
  tve += ve
  print("PC%d - Variance explained: %7.4f - Total Variance: %7.4f" % (i+1, ve, tve))

## keep 32 principal components, since we get a total explained variance of 90.13%.
X_pca = pca.transform(user_profiles)[:, :32]

## see dataframe
user_profiles_train = pd.DataFrame(data=X_pca, columns=['PCA'+str(i) for i in range(1, X_pca.shape[1]+1)], index=dataset_demo['id_survey'])
user_profiles_train

PC1 - Variance explained:  0.1056 - Total Variance:  0.1056
PC2 - Variance explained:  0.0998 - Total Variance:  0.2053
PC3 - Variance explained:  0.0698 - Total Variance:  0.2751
PC4 - Variance explained:  0.0646 - Total Variance:  0.3397
PC5 - Variance explained:  0.0605 - Total Variance:  0.4002
PC6 - Variance explained:  0.0487 - Total Variance:  0.4488
PC7 - Variance explained:  0.0457 - Total Variance:  0.4946
PC8 - Variance explained:  0.0417 - Total Variance:  0.5363
PC9 - Variance explained:  0.0354 - Total Variance:  0.5717
PC10 - Variance explained:  0.0330 - Total Variance:  0.6048
PC11 - Variance explained:  0.0303 - Total Variance:  0.6351
PC12 - Variance explained:  0.0272 - Total Variance:  0.6624
PC13 - Variance explained:  0.0253 - Total Variance:  0.6877
PC14 - Variance explained:  0.0229 - Total Variance:  0.7106
PC15 - Variance explained:  0.0201 - Total Variance:  0.7308
PC16 - Variance explained:  0.0182 - Total Variance:  0.7490
PC17 - Variance explained:  0.016

,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6,PCA7,PCA8,PCA9,PCA10,...,PCA23,PCA24,PCA25,PCA26,PCA27,PCA28,PCA29,PCA30,PCA31,PCA32
id_survey,,,,,,,,,,,,,,,,,,,,,
1,-0.239667,-0.724184,0.176104,-0.095254,-0.954314,-0.419063,-0.840796,0.007551,0.481188,-0.324011,...,0.147695,-0.090725,-0.183896,0.030945,-0.100360,-0.167151,0.031905,0.054454,0.015850,0.042328
2,0.530840,-0.650158,-0.583168,0.105038,-0.099289,-0.827529,-0.574866,-0.691254,-0.282457,0.669379,...,0.192264,0.003910,0.047016,0.152347,-0.173510,0.122553,0.000411,0.298413,-0.024140,0.107159
3,-0.080826,-0.484748,-0.376392,-0.179945,0.640624,-1.181752,-0.037623,0.074213,0.018961,-0.353339,...,-0.009840,0.052912,0.048498,-0.022751,0.079351,-0.043293,-0.129478,-0.058113,-0.147010,-0.046042
4,-1.142395,0.645292,0.219040,0.363866,-0.156510,-0.810039,0.413194,-0.373339,-0.101481,0.017018,...,-0.028785,0.041891,0.053604,-0.135786,0.070335,0.070865,-0.181348,-0.089977,-0.069628,-0.028364
5,-0.416728,0.158678,-0.751061,0.818306,-0.662683,-0.388436,1.196128,0.280349,-0.885230,0.386610,...,-0.055174,0.002446,-0.365803,-0.441370,-0.038548,-0.347496,0.230320,0.061086,-0.175999,0.210768
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,-0.016200,-0.658585,-0.324948,1.235203,0.552353,-0.277637,0.071325,0.021721,0.088316,-0.047628,...,-0.271565,-0.048871,0.007205,-0.000536,-0.092250,0.015777,-0.125195,0.006038,0.046229,-0.009410
160,1.129883,-0.312828,0.979631,0.015209,-0.767189,-0.348382,0.427490,0.034927,0.285973,1.116673,...,-0.383398,-0.010163,-0.024923,-0.138937,0.210837,0.098871,-0.098460,0.042132,-0.000490,-0.003827
161,-0.185630,1.012813,0.188867,0.737044,-0.593167,0.204533,0.185415,-0.175990,0.457839,0.665888,...,0.066997,-0.289278,0.058178,0.029611,0.269152,-0.142560,0.206242,-0.219477,-0.034279,-0.223099


### Splitting

In [10]:
def check_items(trainset, testset):
  """
  Checks if all items present in testset are also present in the trainset.
  
  Args:
    trainset (list of tuples): Each tuple represents a user-item interaction in the trainset. 
      Each tuple is expected to have the form (user, item, rating).
    testset (list of tuples): Each tuple represents a user-item interaction in the testset. 
      Each tuple is expected to have the form (user, item, rating).
    
  Returns:
    bool: True if all items in the testset are present in the trainset, otherwise returns False
  
  """
  trainset_items = set(list(item for _, item, _ in trainset))
  testset_items = set(list(item for _, item, _ in testset))

  items_unknown = list(testset_items - trainset_items)

  return len(items_unknown) == 0

def split_emorecsys(dataset, train=0.8, test=0.2):
  """
  Splits the dataset into training and testing sets, ensuring that each user's ratings are split accordingly.
  The function guarantees that each testset contains at least one relevant rating (liked item) and ensures
  that all items in the testset also appear in the trainset.

  Args:
    dataset (pandas.DataFrame): Dataframe containing user ratings. The dataframe must have columns `id_survey` (user ID),
      `id_photo` (item ID), and `like_bool` (binary rating indicating whether the user likes the item or not).
    train (float): The proportion of the dataset to include in the trainset. Default if 0.8.
    test (float): The proportion of the dataset to include in the testset. Default if 0.2.

  Returns:
    surprise_train (surprise.Trainset): A trainset formatted for use with the Surprise library.
    testset (list of tuples): A list of tuples representing the testset, where each tuple has the form (user, item, rating).
  
  """
  trainset, testset = list(), list()

  items_ratings = dataset.groupby('id_photo').size().to_dict() # get number of rating for each id_photo
  items_one_rating = [id_photo for id_photo, size in items_ratings.items() if size == 1] # get the ones with only 1 rating
  # print(items_one_rating)


  for user in list(set(dataset['id_survey'])):
    user_ratings = dataset[dataset['id_survey'] == user]
    all_ratings = list((user, id_photo, like_bool) for id_photo, like_bool in zip(user_ratings['id_photo'], user_ratings['like_bool']))
    size_ratings = len(all_ratings)

    user_trainset = list(rating for rating in all_ratings if rating[1] in items_one_rating) # start trainset with items with only one rating

    relevant = list(x for x in all_ratings if x[2] == 1)
    if relevant: # making sure that we have at least one relevant rating in the testset
      new_rating = random.sample(relevant, 1)
      user_testset = new_rating if new_rating[0] not in user_trainset else list()
    else:
      raise ValueError('The user ' + str(user) + ' did not liked any photo')

    while len(user_testset) < size_ratings*test: # add random ratings until fullfil the size of testset
      new_rating = random.choice(all_ratings)
      if new_rating not in user_testset and new_rating not in user_trainset:
        user_testset.append(new_rating)

    # adding remaining ratings to trainset
    user_trainset.extend(new_rating for new_rating in all_ratings if new_rating not in user_testset and new_rating not in user_trainset)

    assert len(user_trainset) == size_ratings*train
    assert len(user_testset) == size_ratings*test

    # print(user_trainset)
    # print(user_testset)

    trainset.extend(user_trainset)
    testset.extend(user_testset)

  if check_items(trainset, testset):
    train_df = pd.DataFrame(trainset, columns=['id_survey', 'id_photo', 'like_bool']) # like_bool
  else:
    return split_emorecsys(dataset, train, test)

  reader = Reader(rating_scale=(0,1)) # like_bool
  surprise_train = Dataset.load_from_df(train_df, reader).build_full_trainset()

  return surprise_train, testset

### Ratings Matrix

In [11]:
dataset_cf = pd.read_csv("../data/csvs/ratings.csv")
dataset_cf.head()

,id,id_photo,id_survey,like_bool,anger,fear,disgust,sadness,happiness,surprise,neutral,valence,arousal,dominance
0,1,1754,1,1,0,0,0,0,0,0,5,3,1,2
1,2,1785,1,1,0,0,0,2,0,0,0,2,2,3
2,3,123,1,0,0,0,0,4,0,0,0,1,3,1
3,4,2315,1,0,0,0,0,2,0,0,0,2,1,3
4,5,1548,1,0,0,0,0,0,0,0,5,3,1,3


In [12]:
dataset_cf = dataset_cf[dataset_cf['id_survey'].isin(user_profiles_train.index.tolist())]
dataset_cf

,id,id_photo,id_survey,like_bool,anger,fear,disgust,sadness,happiness,surprise,neutral,valence,arousal,dominance
0,1,1754,1,1,0,0,0,0,0,0,5,3,1,2
1,2,1785,1,1,0,0,0,2,0,0,0,2,2,3
2,3,123,1,0,0,0,0,4,0,0,0,1,3,1
3,4,2315,1,0,0,0,0,2,0,0,0,2,1,3
4,5,1548,1,0,0,0,0,0,0,0,5,3,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2440,2441,2546,163,0,0,5,0,5,0,0,0,1,5,1
2441,2442,2097,163,0,0,0,3,0,5,0,0,2,2,1
2442,2443,1898,163,0,0,0,3,5,0,1,0,1,4,1
2443,2444,1871,163,0,0,0,0,5,0,0,0,1,1,1


In [13]:
ratings_matrix_like = dataset_cf.pivot_table(index='id_survey', columns='id_photo', values='like_bool').reset_index(drop=True)

## 3. Modelling and Evaluating

In [14]:
models_grid = {'KMeans': {'n_clusters': np.arange(5, int(math.ceil(user_profiles_train.shape[0]/6))+1, 2),
                          'init': ['k-means++', 'random'],
                          'n_init': np.arange(5, int(math.ceil(user_profiles_train.shape[0]/6))+1, 2),
                          'algorithm': ['lloyd', 'elkan']},

               'AgglomerativeClustering': {'n_clusters': np.arange(5, int(math.ceil(user_profiles_train.shape[0]/6))+1, 2),
                                           'metric': ['euclidean', 'l1', 'l2', 'manhattan', 'cosine'],
                                           'linkage': ['ward', 'complete', 'average']},

               'SpectralClustering': {'n_clusters': np.arange(5, 31, 5),
                                      'affinity': ['rbf', 'nearest_neighbors'],
                                      'gamma': np.arange(0.4, 3.1, 0.5),
                                      'n_neighbors': np.arange(5, 31, 5),
                                      'eigen_solver': ['arpack', 'lobpcg'],
                                      'assign_labels': ['kmeans', 'discretize']},

               'DBSCAN': {'eps': np.arange(0.007, 0.1, 0.007), # np.linspace(0.001, 0.1, num=100),
                          'min_samples': np.arange(1, 5),
                          'metric': ['euclidean', 'manhattan']}}

##### Evaluation Functions

In [15]:
def similar_users(user, model, n=5, model_dist=None, Kmeans=False):
  """
  Identifies the most similar users to a given test user based on their profiles. It can use clustering models and distance
  measures to find similar users.

  Args:
    user (int): The ID of the test user for whom similar users are to be found.
    model (object): A clustering model that has been fitted to user profiles.
    n (int): The number of similar users to return. Default is 5.
    model_dist (array, optional): Distance matrix for cluster centers. Required if Kmeans is True.
    Kmeans (bool): A flag indicating whether to use KMeans clustering for finding similar users. Default is False.

  Returns:
    users_similar (list of tuples): A list of tuples where each tuple contains a user ID and the similarity score. The list 
    is sorted in descendin order of similarity.

  """
  ## 1. buscar o cluster do user test
  test_user = user_profiles_train.loc[user]
  user_idx = user_profiles_train.index.get_loc(user)

  ## 2. buscar os users do mesmo cluster do user test
  user_cluster = model.labels_[user_idx]
  users = [id for id, cluster in enumerate(model.labels_) if cluster == user_cluster and id != user_idx]

  if len(users) < 5: # 2.5. vamos buscar mais users
    if Kmeans:
      user_dist = model_dist[user_profiles_train.index.get_loc(user)]
      sim_cluster = sorted(range(len(user_dist)), key=lambda x: user_dist[x])[1:2][0] # vamos só buscar 1 cluster extra
      users.extend([id for id, cluster in enumerate(model.labels_) if cluster == sim_cluster])

    else:
      dist = euclidean_distances([test_user], user_profiles_train.values)[0] # aqui vamos ver todos os users
      sorted_users = sorted(range(len(dist)), key=lambda x: dist[x])

      # filtrar para não haver repetição de users, ou inserção do test_user
      sorted_users_new = [x for x in sorted_users if x not in users]

      for user in sorted_users_new:
        if len(users) < 5 and user != user_idx:
          users.append(user)

  ## 3. calcular semelhanças entre o user test e os users dos clusters mais próximos
  users_similar = []
  for user_id in users:
      user_profile = user_profiles_train.iloc[user_id]
      similarity_score = 1 / (1 + euclidean_distances([test_user], [user_profile])[0])
      users_similar.append((user_id, similarity_score))

  ## 4. ordem decrescente para o valor de semelhança
  users_similar = sorted(users_similar, key=lambda x: x[1], reverse=True)

  return users_similar[:n]

In [ ]:
def precision_recall_at_k(recommended, user_test, k=10):
  """
  Calculates the precision, recall and F1 scores at specified cutoff k for a given set of recommended items compared to user's
  test set.

  Args:
    recommended (list): A list of recommended item IDs.
    user_test (list of tuples): A list of tuples representing user0s test interaction, where each tuple contains (user, item, rating).
    k (int): The cutoff for the number of top recommendations to consider. Default is 10.
    
  Returns:
    precision (float): The precision score at k.
    recall (float): The recall score at k.
    f1score (float): The F1 score at k.

  """
  relevant_items = list(item[1] for item in user_test if item[2] > 0)

  rel = len(relevant_items) # total number of relevant items to the user
  rel_rec = np.sum(np.isin(recommended[:k], relevant_items)) # number of relevant items recommended to the user

  # k is the total number of recommended items to the user
  precision = rel_rec / k # number of relevant items recommended to the user / total number of recommended items to the user
  recall = rel_rec / rel if rel != 0 else 1  # number of relevant items recommended to the user / total number of relevant items to the user
  f1score = (2*precision*recall) / (precision+recall) if (precision+recall) != 0 else 0.0

  return precision, recall, f1score

def evaluation(ratings_matrix, user, rec_users, user_test, k=10):
  """
  Evaluates the recommendation model by computing precision, recall, and F1 scores for a given user and their similar users'
  ratings, weighted by similarity.

  Args:
    ratngs_matrix (pandas.DataFrame): A matrix where rows represent users and columns represent items, with ratings as values.
    user (int): The ID of the user for whom the recommendations are evaluated.
    rec_users (list of tuples): A list of tuples where each tuple contains a similar user's ID and their similarity score.
    user_test (list of tuples):  A list of tuples representing the user's test interactions, where each tuple contains (user, item, rating).
    k (int): The number of top recommendations to consider. Default is 10.

  Returns:
    precisions (list of floats): Precision scores at each k.
    recalls (list of floats): Recall scores at each k.
    f1scores (list of floats): F1 scores at each k.

  """
  weights = [1 / (i+1) for i, _ in enumerate(rec_users)]

  users = list(user[0] for user in rec_users)
  rec_users_items = ratings_matrix.loc[users] # items of similar users

  item_count = rec_users_items.astype(bool).sum(axis=0) # number of ratings of each item within the similar users
  item_count = item_count.replace(0, 1) # if any item has 0 rating within the similar users, it will have count 1 to avoid zero divisions

  # for each line (each user) it's applied the weighted attributed to that user, so top 1 > top 2 > top 3 > etc.
  weighted_scores = rec_users_items.multiply(weights, axis=0).sum(axis=0) / item_count

  recommended_items = weighted_scores.nlargest(k).index

  precisions, recalls, f1scores = [], [], []

  # for i in range(k, k+1):
  for i in range(1, k+1):
    precision, recall, f1score = precision_recall_at_k(recommended_items, user_test, i)
    # print("K =", i, "- Precision:", precision, ", Recall:", recall, ", F1 Score:", f1score)

    precisions.append(precision)
    recalls.append(recall)
    f1scores.append(f1score)

  return precisions, recalls, f1scores

def avg_metrics(precisions, recalls, f1scores, k):
  """
  Calculates the average precision, recall, and F1 scores across multiple runs for different values of k.

  Args:
    precisions (list of lists): A list of precisions scores for each run.
    recalls (list of lists): A list of recalls scores for each run.
    f1scores (list of lists): A list of F1 scores for each run.
    k (int): The maximum value of k for which the metrics are calculated.

  Returns:
    precisions_avg (list of floats): Average precision scores for each k.
    recalls_avg (list of floats): Average recall scores for each k.
    f1scores_avg (list of floats): Average F1 scores for each k.

  """
  precisions_avg, recall_avg, f1score_avg = [], [], []
  
  for i in range(k):
    precision, recall, f1score = [], [], []

    for prec, rec, f1 in zip(precisions, recalls, f1scores):
      precision.append(prec[i])
      recall.append(rec[i])
      f1score.append(f1[i])

    precisions_avg.append(np.round(np.mean(precision), 4))
    recall_avg.append(np.round(np.mean(recall), 4))
    f1score_avg.append(np.round(np.mean(f1score), 4))

  return precisions_avg, recall_avg, f1score_avg

def gridsearch_recommendation(model, param_grid, ratings_matrix, dataset, k=10):
  """
  Performs a grid search over hyperparameters for a given recommendation model to find the best parameters based on F1 score at k.

  Args:
    model (class): The recommendation model class to be used.
    param_grid (dict): A dictionary with hyperparameters and their respective values to be searched.
    ratings_matrix (pandas.DataFrame): A matrix where rows represent users and columns represent items, with ratings as values.
    dataset (pandas.DataFrame): The dataset containing user-item interaction with columns `id_survey`, `id_photo`, and `like_bool`.
    k (int): The number of top recommendations to consider. Default is 10.
  
  """
  best_score = -1
  best_params = None

  for params in list(ParameterGrid(param_grid)):
    if (AgglomerativeClustering == model) and (params['linkage'] == 'ward' and not params['metric'] == 'euclidean'):
      pass
    else:
      algo = model(**params)
      model_dist = None

      if KMeans == model:
        model_dist = algo.fit_transform(user_profiles_train)
      else:
        algo.fit(user_profiles_train)

      precisions_final, recalls_final, f1scores_final = [], [], []
      for i in range(5):
        trainset, testset = split_emorecsys(dataset)
        test_users = list(set(item[0] for item in testset))

        for user in test_users:
          user_relevant = list(item for item in testset if item[0] == user)
          rec_users = similar_users(user=user, model=algo, model_dist=model_dist, Kmeans=(KMeans == model))
          precisions, recalls, f1scores = evaluation(ratings_matrix, user, rec_users, user_relevant, k=k)

          precisions_final.append(precisions)
          recalls_final.append(recalls)
          f1scores_final.append(f1scores)

      precision_avg, recall_avg, f1score_avg = avg_metrics(precisions_final, recalls_final, f1scores_final, k)

      if f1score_avg[-1] > best_score:
        best_score = f1score_avg[-1]
        best_params = params

        # keeping the precision, recall and f1score values for the best model
        best_precisions, best_recalls, best_f1scores = precision_avg, recall_avg, f1score_avg

  print(f'Best parameters: {best_params}')
  print(f'Best F1 Score at K = 10: {best_score}\n')

  for i in range(k):
    print(f"K = {i+1} - Precision: {best_precisions[i]}, Recall: {best_recalls[i]}, F1 Score: {best_f1scores[i]}")

### 3.1. KMeans

In [ ]:
gridsearch_recommendation(KMeans, models_grid['KMeans'], ratings_matrix_like, dataset_cf, k=10)

Best parameters: {'algorithm': 'elkan', 'init': 'random', 'n_clusters': 19, 'n_init': 19}
Best F1 Score at K = 10: 0.0352

K = 1 - Precision: 0.0245, Recall: 0.0121, F1 Score: 0.0145
K = 2 - Precision: 0.0227, Recall: 0.0211, F1 Score: 0.0205
K = 3 - Precision: 0.0266, Recall: 0.0368, F1 Score: 0.0296
K = 4 - Precision: 0.0264, Recall: 0.0485, F1 Score: 0.0329
K = 5 - Precision: 0.0263, Recall: 0.0593, F1 Score: 0.0352
K = 6 - Precision: 0.0243, Recall: 0.0656, F1 Score: 0.0345
K = 7 - Precision: 0.0244, Recall: 0.0783, F1 Score: 0.0362
K = 8 - Precision: 0.0225, Recall: 0.0824, F1 Score: 0.0346
K = 9 - Precision: 0.0228, Recall: 0.0955, F1 Score: 0.036
K = 10 - Precision: 0.0218, Recall: 0.101, F1 Score: 0.0352


### 3.2. AgglomerativeClustering

In [ ]:
gridsearch_recommendation(AgglomerativeClustering, models_grid['AgglomerativeClustering'], ratings_matrix_like, dataset_cf, k=10)

Best parameters: {'linkage': 'average', 'metric': 'euclidean', 'n_clusters': 21}
Best F1 Score at K = 10: 0.0335

K = 1 - Precision: 0.0147, Recall: 0.0072, F1 Score: 0.0084
K = 2 - Precision: 0.0172, Recall: 0.0155, F1 Score: 0.0155
K = 3 - Precision: 0.018, Recall: 0.0235, F1 Score: 0.0197
K = 4 - Precision: 0.0209, Recall: 0.0391, F1 Score: 0.0263
K = 5 - Precision: 0.0211, Recall: 0.0489, F1 Score: 0.0286
K = 6 - Precision: 0.0211, Recall: 0.0607, F1 Score: 0.0305
K = 7 - Precision: 0.02, Recall: 0.0671, F1 Score: 0.0301
K = 8 - Precision: 0.0207, Recall: 0.0777, F1 Score: 0.032
K = 9 - Precision: 0.0213, Recall: 0.089, F1 Score: 0.0337
K = 10 - Precision: 0.0207, Recall: 0.0961, F1 Score: 0.0335


### 3.3. SpectralClustering

In [ ]:
gridsearch_recommendation(SpectralClustering, models_grid['SpectralClustering'], ratings_matrix_like, dataset_cf, k=10)

/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_spectral_embedding.py:393: UserWarning: Exited postprocessing with accuracies 
[9.63123883e-14 5.25913712e-07 5.00441298e-08 7.59276151e-07
 5.31056338e-07 2.46494188e-06]
not reaching the requested tolerance 2.428889274597168e-06.
  _, diffusion_map = lobpcg(


Best parameters: {'affinity': 'nearest_neighbors', 'assign_labels': 'kmeans', 'eigen_solver': 'lobpcg', 'gamma': 2.9, 'n_clusters': 20, 'n_neighbors': 30}
Best F1 Score at K = 10: 0.0378

K = 1 - Precision: 0.0196, Recall: 0.0125, F1 Score: 0.0129
K = 2 - Precision: 0.0202, Recall: 0.0219, F1 Score: 0.0193
K = 3 - Precision: 0.0225, Recall: 0.0344, F1 Score: 0.0258
K = 4 - Precision: 0.0242, Recall: 0.0497, F1 Score: 0.0312
K = 5 - Precision: 0.0258, Recall: 0.064, F1 Score: 0.0355
K = 6 - Precision: 0.0252, Recall: 0.0734, F1 Score: 0.0364
K = 7 - Precision: 0.0235, Recall: 0.0785, F1 Score: 0.0352
K = 8 - Precision: 0.0244, Recall: 0.092, F1 Score: 0.0377
K = 9 - Precision: 0.0239, Recall: 0.1006, F1 Score: 0.0378
K = 10 - Precision: 0.0233, Recall: 0.1092, F1 Score: 0.0378


### 3.4. DBSCAN

In [ ]:
gridsearch_recommendation(DBSCAN, models_grid['DBSCAN'], ratings_matrix_like, dataset_cf, k=10)

Best parameters: {'eps': 0.056, 'metric': 'euclidean', 'min_samples': 1}
Best F1 Score at K = 10: 0.0346

K = 1 - Precision: 0.0221, Recall: 0.0104, F1 Score: 0.0129
K = 2 - Precision: 0.0209, Recall: 0.0186, F1 Score: 0.0186
K = 3 - Precision: 0.0196, Recall: 0.0262, F1 Score: 0.0215
K = 4 - Precision: 0.0233, Recall: 0.0423, F1 Score: 0.0291
K = 5 - Precision: 0.0231, Recall: 0.0505, F1 Score: 0.0309
K = 6 - Precision: 0.0235, Recall: 0.062, F1 Score: 0.0333
K = 7 - Precision: 0.0228, Recall: 0.0722, F1 Score: 0.0338
K = 8 - Precision: 0.0218, Recall: 0.0798, F1 Score: 0.0335
K = 9 - Precision: 0.0213, Recall: 0.0869, F1 Score: 0.0335
K = 10 - Precision: 0.0215, Recall: 0.0982, F1 Score: 0.0346


## 4. Recommending

In [17]:
dataset_photos = pd.read_csv("../data/csvs/photos.csv")
dataset_photos.set_index('id', inplace=True)

In [23]:
def display_image(show_list, n_show=3):
  """
  Displays a list of images in a grid format. It reads images from the specified file paths and displays them using widgets.

  Args:
    show_list (list of str): A list of item IDs representing the images to be displayed.
    n_show (int): The number of images to display per row. Default is 3.

  """
  relevant_images_widgets = []
  for item in show_list:
    image = dataset_photos.loc[item]['file_name']
    ext = dataset_photos.loc[item]['ext']
    image_path = f'../data/photos/{image}.{ext}'

    with open(image_path, "rb") as file:
      img = file.read()

    img_widget = WidgetImage(value=img, format='jpg', width=200, height=200)
    relevant_images_widgets.append(img_widget)

  display(VBox([HBox(relevant_images_widgets[i:i+n_show]) for i in range(0, len(relevant_images_widgets), n_show)]))

In [24]:
def recommending_demo(model, params, dataset):
  """
  Demonstrates a recommendation system by selecting a random user from the testset, displaying their
  relevant images, and showing the top 5 recommended images based on the similarity with other users.

  Args:
    model (class): The recommendation model class to be used.
    params (dict): A dictionary of parameters to initialize the model.
    dataset (pandas.DataFrame): The dataset containing user-item interaction with columns `id_survey`, `id_photo`, 
      and `like_bool`.
  
  """
  trainset, testset = split_emorecsys(dataset)

  test_users = list(set(item[0] for item in testset))
  user = random.choice(test_users)
  items_relevant = list(item[1] for item in testset if item[0] == user and item[2] > 0)

  print('RELEVANT IMAGES:')
  display_image(items_relevant, len(items_relevant))

  algo = model(**params)
  algo.fit(user_profiles_train)

  rec_users = similar_users(user=user, model=algo, model_dist=None, Kmeans=False)
  weights = [1 / (i+1) for i, _ in enumerate(rec_users)]

  users = list(user[0] for user in rec_users)
  rec_users_items = ratings_matrix_like.loc[users]
  item_count = rec_users_items.astype(bool).sum(axis=0)
  item_count = item_count.replace(0, 1)

  weighted_scores = rec_users_items.multiply(weights, axis=0).sum(axis=0) / item_count

  recommended_items = weighted_scores.nlargest(5).index

  print('TOP 5 IMAGES RECOMMENDED:')
  display_image(recommended_items, 5)

In [30]:
recommending_demo(SpectralClustering, {'affinity': 'nearest_neighbors', 'assign_labels': 'kmeans', 'eigen_solver': 'lobpcg', 'gamma': 2.9, 'n_clusters': 20, 'n_neighbors': 30}, dataset_cf)

RELEVANT IMAGES:


TOP 5 IMAGES RECOMMENDED:
